In [ ]:
#加载飞桨、NumPy和相关类库
import paddle
from paddle.nn import Linear
import paddle.nn.functional as F
import numpy as np
import os
import random

: 

# 数据导入模块  
包括数据整形  归一化处理  测试集和评价集划分

In [21]:
def load_data():
    # 从文件导入数据
    datafile = 'housing.data'
    data = np.fromfile(datafile, sep=' ', dtype=np.float32)

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 计算train数据集的最大值，最小值
    maximums, minimums = training_data.max(axis=0), training_data.min(axis=0)
  
    # 记录数据的归一化参数，在预测时对数据做归一化
    global max_values
    global min_values
   
    max_values = maximums
    min_values = minimums
  
    # 对数据进行归一化处理
    for i in range(feature_num):
        data[:, i] = (data[:, i] - min_values[i]) / (maximums[i] - minimums[i])


 # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data


In [26]:
# 验证数据集读取程序的正确性
training_data, test_data = load_data()
print(training_data.shape)
print(training_data[1,:])
print(test_data.shape)
print(test_data[1,:])

(404, 14)
[2.35922547e-04 0.00000000e+00 2.62405723e-01 0.00000000e+00
 1.72839552e-01 5.47997713e-01 7.82698274e-01 3.48961979e-01
 4.34782617e-02 1.14822544e-01 5.53191364e-01 1.00000000e+00
 2.04470202e-01 3.68888885e-01]
(102, 14)
[0.76334244 0.         0.7002779  0.         0.6337449  0.4065913
 1.         0.02689848 1.         1.         0.8085107  0.96341616
 0.5863686  0.        ]


# 定义模型
包含网络结构
前向计算模块

In [15]:
class Regressor(paddle.nn.Layer):

    # self代表类的实例自身
    def __init__(self):
        # 初始化父类中的一些参数
        super(Regressor, self).__init__()
  
        # 定义一层全连接层，输入维度是13，输出维度是1
        self.fc = Linear(in_features=13, out_features=1)
  
    # 网络的前向计算
    def forward(self, inputs):
        x = self.fc(inputs)
        return x

# 模型声明，模型状态训练开启
关于模型的声明，通过参数调用，模型训练的开启以及学习率设置

In [23]:
# 声明定义好的线性回归模型
model = Regressor()
# 开启模型训练模式
model.train()
# 定义优化算法，使用随机梯度下降SGD
# 学习率设置为0.01
opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())

# 训练过程
1包含训练数据循环次数和数据块大小，转化张量
2调用模型训练
3损失函数计算
4优化函数
在PaddlePaddle中，backward()方法的具体使用取决于你的需求：

如果你有一个标量（scalar）结果，比如一个损失函数的值，你可以直接对这个标量调用backward()，PaddlePaddle会自动计算整个网络的梯度。

如果你有一个非标量的输出，比如一个向量或矩阵，你需要首先用paddle.mean()或其他方法将其归约为一个标量，然后再调用backward()。

你也可以直接对一个中间变量（比如网络中的某个隐藏层的输出）调用backward()，但这通常不是整个网络的损失函数，所以只有在特殊情况下才会这样做。



In [24]:
EPOCH_NUM = 10   # 设置外层循环次数
BATCH_SIZE = 10  # 设置batch大小

# 定义外层循环
for epoch_id in range(EPOCH_NUM):
    # 在每轮迭代开始之前，将训练数据的顺序随机的打乱
    np.random.shuffle(training_data)
    # 将训练数据进行拆分，每个batch包含10条数据
    mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
    # 定义内层循环
    for iter_id, mini_batch in enumerate(mini_batches):
        x = np.array(mini_batch[:, :-1]) # 获得当前批次训练数据
        y = np.array(mini_batch[:, -1:]) # 获得当前批次训练标签（真实房价）
        # 将numpy数据转为飞桨动态图tensor的格式
        house_features = paddle.to_tensor(x)
        prices = paddle.to_tensor(y)
  
        # 前向计算
        predicts = model(house_features)
  
        # 计算损失
        loss = F.square_error_cost(predicts, label=prices)
        avg_loss = paddle.mean(loss)
        if iter_id%20==0:
            print("epoch: {}, iter: {}, loss is: {}".format(epoch_id, iter_id, avg_loss.numpy()))
  
        # 反向传播，计算每层参数的梯度值，backward()是paddlepaddle框架中Tensor类的方法，调用一个tensor的backward方法
        avg_loss.backward()
        # 更新参数，根据设置好的学习率迭代一步
        opt.step()
        # 清空梯度变量，以备下一轮计算
        opt.clear_grad()

epoch: 0, iter: 0, loss is: 0.9277139902114868
epoch: 0, iter: 20, loss is: 0.3200933337211609
epoch: 0, iter: 40, loss is: 0.18067634105682373
epoch: 1, iter: 0, loss is: 0.22659127414226532
epoch: 1, iter: 20, loss is: 0.18577122688293457
epoch: 1, iter: 40, loss is: 0.24033069610595703
epoch: 2, iter: 0, loss is: 0.1494569480419159
epoch: 2, iter: 20, loss is: 0.04799516871571541
epoch: 2, iter: 40, loss is: 0.1554272323846817
epoch: 3, iter: 0, loss is: 0.1545664370059967
epoch: 3, iter: 20, loss is: 0.07703548669815063
epoch: 3, iter: 40, loss is: 0.2267395257949829
epoch: 4, iter: 0, loss is: 0.1393933743238449
epoch: 4, iter: 20, loss is: 0.09830181300640106
epoch: 4, iter: 40, loss is: 0.11449439823627472
epoch: 5, iter: 0, loss is: 0.06901063770055771
epoch: 5, iter: 20, loss is: 0.1642816811800003
epoch: 5, iter: 40, loss is: 0.13635900616645813
epoch: 6, iter: 0, loss is: 0.10634420812129974
epoch: 6, iter: 20, loss is: 0.05928543210029602
epoch: 6, iter: 40, loss is: 0.0418

In [25]:
# 保存模型参数，文件名为LR_model.pdparams
paddle.save(model.state_dict(), 'LR_model.pdparams')
print("模型保存成功，模型参数保存在LR_model.pdparams中")

模型保存成功，模型参数保存在LR_model.pdparams中


In [27]:
def load_one_example():
    # 从上边已加载的测试集中，随机选择一条作为测试数据
    idx = np.random.randint(0, test_data.shape[0])
    idx = -10
    one_data, label = test_data[idx, :-1], test_data[idx, -1:]
    # 修改该条数据shape为[1,13]
    one_data =  one_data.reshape([1,13])

    return one_data, label

In [28]:
# 参数为保存模型参数的文件地址
model_dict = paddle.load('LR_model.pdparams')
model.load_dict(model_dict)
model.eval()  # 参数为数据集的文件地址


one_data, label = load_one_example()
# 将数据转为动态图的variable格式 
one_data = paddle.to_tensor(one_data)
predict = model(one_data)

# 对结果做反归一化处理
predict = predict * (max_values[-1] - min_values[-1]) + min_values[-1]
# 对label数据做反归一化处理
label = label * (max_values[-1] - min_values[-1]) + min_values[-1]

print("Inference result is {}, the corresponding label is {}".format(predict.numpy(), label))

Inference result is [[20.891407]], the corresponding label is [19.7]
